# Introduction to Semantic Kernel in Python

With chat completion, you can simulate a back-and-forth conversation with an AI agent. This is of course useful for creating chat bots, but it can also be used for creating autonomous agents that can complete business processes, generate code, and more. As the primary model type provided by OpenAI, Google, Mistral, Facebook, and others, chat completion is the most common AI service that you will add to your Semantic Kernel project.

When picking out a chat completion model, you will need to consider the following:

* What modalities does the model support (e.g., text, image, audio, etc.)?
* Does it support function calling?
* How fast does it receive and generate tokens?
* How much does each token cost?

## Creating the required Azure resources

You can run the powershell script `create-azure-ai-resources.ps1` to create the following resources:
* Azure AI Services with Hub and Project
* GPT-4o model

## Installing the necessary packages

Import Semantic Kernel SDK from pypi.org

In [30]:
%pip install -U semantic-kernel --quiet

Note: you may need to restart the kernel to use updated packages.


Check the current version of Semantic Kernel.

In [31]:
from semantic_kernel import __version__

__version__

'1.22.0'

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

>NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.

#### Using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

>Tip: There are three methods to supply the required information to AI services. You may either provide the information directly through the constructor, set the necessary environment variables, or create a .env file within your project directory containing the environment variables.

Let's load the environment variables.

In [32]:
from dotenv import load_dotenv
import os

if os.path.exists(".env"):
    load_dotenv(override=True)

## Creating a chat completion service

>Note: The AzureChatCompletion service also supports Microsoft Entra authentication. If you don't provide an API key, the service will attempt to authenticate using the Entra token.

In [6]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    service_id="default"
)

You can start using the completion service right away or add the chat completion service to a kernel. You can use the following code to add a service to the kernel.

In [17]:
from semantic_kernel import Kernel

kernel = Kernel()

kernel.add_service(chat_completion)

>TIP: Adding the chat completion service to the kernel is not required if you don't need to use other services in the kernel. You can use the chat completion service directly in your code.

## Retrieving chat completion services

Once you've added chat completion services to your kernel, you can retrieve them using the get service method. Below is an example of how you can retrieve a chat completion service from the kernel.

In [21]:
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase

# Retrieve the chat completion service by id
chat_completion = kernel.get_service(service_id="default")


## Using chat completion services

Now that you have a chat completion service, you can use it to generate responses from an AI agent. There are two main ways to use a chat completion service:

* Non-streaming: You wait for the service to generate an entire response before returning it to the user.
* Streaming: Individual chunks of the response are generated and returned to the user as they are created.

Before getting started, you will need to manually create an execution settings instance to use the chat completion service if you did not register the service with the kernel.

In [20]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.contents.chat_history import ChatHistory

execution_settings = AzureChatPromptExecutionSettings()

Below are the two ways you can use a chat completion service to generate responses.

## Non-streaming chat completion

To use non-streaming chat completion, you can use the following code to generate a response from the AI agent.

In [22]:
system_message = "You are a helpful assistant that can answer questions and provide information."

chat_history = ChatHistory(system_message=system_message)

chat_history.add_user_message("Hello, how are you?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

print(response)

Hello! I'm here and ready to help you with any questions or information you need. How can I assist you today?


## Streaming chat completion

To use streaming chat completion, you can use the following code to generate a response from the AI agent.

In [29]:
chat_history = ChatHistory()
chat_history.add_user_message("Hello, how are you? Answer in 20 sentences.")

response = chat_completion.get_streaming_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

async for chunk in response:
    print(chunk, end=".")

.Hello.!. I'm. just. a. virtual. assistant.,. so. I. don't. have. feelings.,. but. I'm. here. and. ready. to. help. you. with. whatever. you. need... While. I. don't. experience. emotions. in. the. way. humans. do.,. I. strive. to. be. as. helpful. and. reliable. as. possible... It's. always. exciting. to. interact. with. people. and. assist. with. their. questions. or. tasks... 

.The. world. is. constantly. evolving.,. and. there's. so. much. happening. at. any. given. time.—.be. it. in. technology.,. science.,. or. even. pop. culture... I. love. to. share. information. about. new. advancements. and. insights. across. a. multitude. of. fields... This. allows. anyone. interacting. with. me. to. gain. a. little. extra. knowledge.,. perhaps. even. spar.king. curiosity..

.You. could. say. I'm. like. a. vast. library. at. your. fingertips.,. offering. information. and. data. on. a. wide. array. of. topics... Whether. you're. interested. in. history.,. music.,. current. events.,. or. even

## More resources

https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion